In [37]:
import os
import act
import xarray as xr
import matplotlib.pyplot as plt
import altair as alt
import pandas as pd

In [41]:

username = os.getenv("ARM_USERNAME")
token = os.getenv("ARM_TOKEN")

ecor_kp = 'guc30ecorS3.b1'
sebs = 'gucsebsS3.b1'
ld = 'gucldM1.b1'
output_dir = os.path.join('/data2/elilouis/sublimationofsnow', ld)

start_date = "2023-02-01"
end_date = "2023-02-13"

In [42]:
files = act.discovery.download_data(
    username,
    token,
    ld,
    start_date,
    end_date,
    output=output_dir
)

[DOWNLOADING] gucldM1.b1.20230211.000000.cdf
[DOWNLOADING] gucldM1.b1.20230213.000000.cdf
[DOWNLOADING] gucldM1.b1.20230212.000000.cdf
[DOWNLOADING] gucldM1.b1.20230201.000000.cdf
[DOWNLOADING] gucldM1.b1.20230202.000000.cdf
[DOWNLOADING] gucldM1.b1.20230203.000000.cdf
[DOWNLOADING] gucldM1.b1.20230204.000000.cdf
[DOWNLOADING] gucldM1.b1.20230205.000000.cdf
[DOWNLOADING] gucldM1.b1.20230206.000000.cdf
[DOWNLOADING] gucldM1.b1.20230207.000000.cdf
[DOWNLOADING] gucldM1.b1.20230208.000000.cdf
[DOWNLOADING] gucldM1.b1.20230209.000000.cdf
[DOWNLOADING] gucldM1.b1.20230210.000000.cdf


In [43]:
files

['/data2/elilouis/sublimationofsnow/gucldM1.b1/gucldM1.b1.20230211.000000.cdf',
 '/data2/elilouis/sublimationofsnow/gucldM1.b1/gucldM1.b1.20230213.000000.cdf',
 '/data2/elilouis/sublimationofsnow/gucldM1.b1/gucldM1.b1.20230212.000000.cdf',
 '/data2/elilouis/sublimationofsnow/gucldM1.b1/gucldM1.b1.20230201.000000.cdf',
 '/data2/elilouis/sublimationofsnow/gucldM1.b1/gucldM1.b1.20230202.000000.cdf',
 '/data2/elilouis/sublimationofsnow/gucldM1.b1/gucldM1.b1.20230203.000000.cdf',
 '/data2/elilouis/sublimationofsnow/gucldM1.b1/gucldM1.b1.20230204.000000.cdf',
 '/data2/elilouis/sublimationofsnow/gucldM1.b1/gucldM1.b1.20230205.000000.cdf',
 '/data2/elilouis/sublimationofsnow/gucldM1.b1/gucldM1.b1.20230206.000000.cdf',
 '/data2/elilouis/sublimationofsnow/gucldM1.b1/gucldM1.b1.20230207.000000.cdf',
 '/data2/elilouis/sublimationofsnow/gucldM1.b1/gucldM1.b1.20230208.000000.cdf',
 '/data2/elilouis/sublimationofsnow/gucldM1.b1/gucldM1.b1.20230209.000000.cdf',
 '/data2/elilouis/sublimationofsnow/gucl

In [55]:
disdro_ds = xr.open_mfdataset(files)

In [56]:
for var in disdro_ds.data_vars:
    print(var)
    print(disdro_ds[var].attrs['long_name'])
    print()


base_time
Base time in Epoch

time_offset
Time offset from base_time

precip_rate
Precipitation intensity

qc_precip_rate
Quality check results on field: Precipitation intensity

weather_code
SYNOP WaWa Table 4680

qc_weather_code
Quality check results on field: SYNOP WaWa Table 4680

equivalent_radar_reflectivity_ott
Radar reflectivity from the manufacturer's software

qc_equivalent_radar_reflectivity_ott
Quality check results on field: Radar reflectivity from the manufacturer's software

number_detected_particles
Number of particles detected

qc_number_detected_particles
Quality check results on field: Number of particles detected

mor_visibility
Meteorological optical range visibility

qc_mor_visibility
Quality check results on field: Meteorological optical range visibility

snow_depth_intensity
New snow height

qc_snow_depth_intensity
Quality check results on field: New snow height

laserband_amplitude
Laserband amplitude

qc_laserband_amplitude
Quality check results on field: Lase

In [57]:
disdro_df = disdro_ds.to_dataframe().reset_index()

In [58]:
disdro_precip_rate_df = pd.DataFrame(
    disdro_df.groupby("time")['precip_rate'].max()
).reset_index()

In [62]:
disdro_precip_rate_30min_df = disdro_precip_rate_df.set_index(
    "time"
).resample(
    "1440Min"
).max().reset_index()

In [63]:
alt.Chart(disdro_precip_rate_30min_df).mark_circle().encode(
    alt.X('time:T'),
    alt.Y("precip_rate:Q")
)

/home/elilouis/mambaforge/envs/arm/lib/python3.11/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [65]:
s = disdro_precip_rate_30min_df.groupby(disdro_precip_rate_30min_df['precip_rate'].ne(0).cumsum())['time'].transform('first')
disdro_precip_rate_30min_df['days_since_precip'] = (disdro_precip_rate_30min_df['time'] - s).dt.days

In [67]:
disdro_precip_rate_30min_df

,time,precip_rate,days_since_precip
0,2023-02-01,0.000000,0
1,2023-02-02,0.000000,1
2,2023-02-03,0.000000,2
3,2023-02-04,0.000000,3
4,2023-02-05,0.123000,0
5,2023-02-06,24.680000,0
6,2023-02-07,0.000000,1
7,2023-02-08,0.934000,0
8,2023-02-09,23.021999,0
9,2023-02-10,0.000000,1


In [79]:
alt.Chart(disdro_precip_rate_30min_df).mark_circle(size=50).encode(
    alt.X('time:T'),
    alt.Y("precip_rate:Q")
) + alt.Chart(disdro_precip_rate_30min_df).mark_line(color='red', strokeDash=[0.5,2]).encode(
    alt.X('time:T'),
    alt.Y("days_since_precip:Q"),
    alt.StrokeDash()
)

alt.LayerChart(...)